In [ ]:
import os
import json
from comparisonHelper import TestParamsReader

In [ ]:
master_file_path = os.path.join(os.getcwd(), "results")
os.makedirs(master_file_path,exist_ok=True)
param_file_name = os.path.join(os.getcwd(), "params.json")

src_path = os.path.join(os.getcwd(), 'src.csv')
tgt_path = os.path.join(os.getcwd(), 'tgt.csv')
runner_results_path = os.path.join(os.getcwd(), 'runner_result.json')

print('Reading testing configs...')
config_reader = TestParamsReader()
all_configs = config_reader.all_configs
total_tests = len(all_configs)
passed_tests = 0
skipped_tests = 0
test_results = []
if all_configs:
    print(f'Running {total_tests} tests...')
    index = 1
    for test in all_configs:
        print(f'Running {index}/{total_tests} {test.name}...', end=' ')
        test_result = {'name': test.name}
        if not test.get_description_errors():
            test.source_config.save_to_csv(src_path)
            test.target_config.save_to_csv(tgt_path)
            # prepare config file
            settings_for_config = {}
            settings_for_config['name'] = test.name
            settings_for_config['source_path'] = src_path
            settings_for_config['target_path'] = tgt_path
            settings_for_config['column_mapping'] = test.column_mapping
            with open(param_file_name, 'w') as f:
                json.dump(settings_for_config, f)
            %run comparison_runner.ipynb
            # read result
            with open(runner_results_path, 'r') as f:
                results = json.load(f)
                result_status = results['status']
                print(result_status)
                test_result['status'] = result_status
                if result_status == 'Passed':
                    passed_tests+=1
        else:
            print('Skipped: ' + '\n'.join(test.get_description_errors()))
            test_result['status'] = 'Skipped'
            skipped_tests+=1
        test_results.append(test_result)
        index+=1
    failed_tests = total_tests - passed_tests - skipped_tests
    print(f'Completed: {passed_tests} passed, {failed_tests} failed, {skipped_tests} skipped')
else:
    print(f'0 tests found, nothing to run')